In [1]:
import nbsetup
import numpy as np
import ergodicpy as ep
from scipy.stats import chi2_contingency
import pandas as pd

In [2]:
def chi2(data):
    chi2, p, dof, expected = chi2_contingency(data)
    result = "Reject H0, they are dependant" if p < 0.05 else "Accept H0, they are independant"
    print('%s (X2=%s,p=%s)' % (result,chi2, p))
    print("")
    print(np.array(data))
    print("")
    print("%s ergodic complexity" % ep.measures(data)['complexity'])
    print(ep.measures(data))
    #return "Dependant" if p < 0.05 else "independant"


def split_ensemble(data, r=1):
    data = np.array(data).copy()
    row = data[r]/2
    data[r] = row
    data = np.append(data, [row], axis=0)
    return data

def double_ensemble(data, r=1):
    data = np.array(data).copy()
    data[r] *= 2
    return data
    

https://helloml.org/an-introduction-to-the-chi-squared-test-with-examples/

A Chi-Square test of independence uses the following null and alternative hypotheses:

- H0: (null hypothesis) The two variables are independent.
- H1: (alternative hypothesis) The two variables are not independent. (i.e. they are associated)

If the p-value that corresponds to the test statistic X2 with (#rows-1)*(#columns-1) degrees of freedom is less than your chosen significance level then you can reject the null hypothesis.

# Making dependant results independant

Let's say we are looking at something trivial such as a childs favourite color (my daughter's favourite is "rainbow" ¯\\\_(ツ)\_/¯).

Here, for the sake of clarity, using the terminology of ergodic complexity - we're going to say that gender represents `ensembles` (i.e. different sample populations), while colour represents `states` of the system.

First let's baseline with a data set where there is no preference of colour between girls and boys. We find that the p-value is > 0.05, so we accept the null hypothesis `H0` and say they are independant.

In [3]:
independant = [
    # Blue, Green, Pink
    [100, 20, 150], # boys
    [110, 18, 142]] # girls

chi2(independant)

Accept H0, they are independant (X2=0.8006317162769938,p=0.6701083534278778)

[[100  20 150]
 [110  18 142]]

0.0017245109787449866 ergodic complexity
{'ensemble': 0.8857665289187394, 'ergodic': 0.8865080918127473, 'divergence': 0.0007415628940078967, 'complexity': 0.0017245109787449866, 'c2': 2.9739381158119915e-06, 'alt': 0.0018315745972641822, 'alt2': 3.3546655053434515e-06}


Now let's take an another ficticous dataset, but this time where there is a preference between girls and boys. Where girls slightly prefer pink. We now find we reject `H0`.

In [4]:
dependant = [
    # Blue, Green, Pink
    [100, 20, 150], # boys
    [125, 30, 290]] # Boys

chi2(dependant)

Reject H0, they are dependant (X2=6.904690895639712,p=0.03167126589457268)

[[100  20 150]
 [125  30 290]]

0.03703540705484103 ergodic complexity
{'ensemble': 0.8438453427023884, 'ergodic': 0.8486397166676904, 'divergence': 0.004794373965301995, 'complexity': 0.03703540705484103, 'c2': 0.0013716213757177685, 'alt': 0.04020272079714195, 'alt2': 0.0016162587594929496}


However, let's say we're unethical and this isn't the result we want. What we can do is split the girls ensemble in two and create an extra ensemble based on some other field we might have collected. Perhaps, girls > 5yrs and girls < 5 yrs.

In [5]:
chi2(split_ensemble(dependant))

Accept H0, they are independant (X2=7.0217029380656095,p=0.13474575970649377)

[[100.   20.  150. ]
 [ 62.   15.  145. ]
 [ 62.5  15.  145. ]]

0.03732739287918715 ergodic complexity
{'ensemble': 0.843545271692169, 'ergodic': 0.8484239681533361, 'divergence': 0.004878696461167182, 'complexity': 0.03732739287918715, 'c2': 0.0013933342591571913, 'alt': 0.04052482922312087, 'alt2': 0.001642261783563111}


We now get the result that we accept `H0` and can say they are independant. Now this third ensemble may not stand up to scrutiny, however that doesn't matter in this case since we're got the hacked result we want. Logic says, if there's independance between girls of different ages and boys, they've certainly got to be independant is you aggregate girls together... right?

### Why did this happen?

It's all about volumes. To illustrate, let's say we have twice the number of girls to begin with because of some samplying bias. We still Reject H0 and say they are dependant.

In [6]:
# double girls row
chi2(double_ensemble(dependant))

Reject H0, they are dependant (X2=8.67118971474828,p=0.013094082644208454)

[[100  20 150]
 [250  60 580]]

0.032424770961447594 ergodic complexity
{'ensemble': 0.8337516114820531, 'ergodic': 0.8374155903356688, 'divergence': 0.003663978853615666, 'complexity': 0.032424770961447594, 'c2': 0.001051365771902335, 'alt': 0.035432875320335594, 'alt2': 0.0012554886534664473}


Now again, let's split the girls ensemble, so that it ends up Accepting H0 and saying it's independant.

In [7]:
# duplicate girls row
chi2(split_ensemble(double_ensemble(dependant)))

Accept H0, they are independant (X2=8.671189714748278,p=0.06986472001847062)

[[100.  20. 150.]
 [125.  30. 290.]
 [125.  30. 290.]]

0.032424770961447594 ergodic complexity
{'ensemble': 0.8337516114820531, 'ergodic': 0.8374155903356688, 'divergence': 0.003663978853615666, 'complexity': 0.032424770961447594, 'c2': 0.001051365771902335, 'alt': 0.035432875320335594, 'alt2': 0.0012554886534664473}


In [8]:
# again for reference
chi2(dependant)

# double and split both
chi2(split_ensemble(split_ensemble(double_ensemble(double_ensemble(dependant),0)),0))

Reject H0, they are dependant (X2=6.904690895639712,p=0.03167126589457268)

[[100  20 150]
 [125  30 290]]

0.03703540705484103 ergodic complexity
{'ensemble': 0.8438453427023884, 'ergodic': 0.8486397166676904, 'divergence': 0.004794373965301995, 'complexity': 0.03703540705484103, 'c2': 0.0013716213757177685, 'alt': 0.04020272079714195, 'alt2': 0.0016162587594929496}
Reject H0, they are dependant (X2=13.809381791279423,p=0.03183948813397328)

[[100.  20. 150.]
 [125.  30. 290.]
 [125.  30. 290.]
 [100.  20. 150.]]

0.03703540705484103 ergodic complexity
{'ensemble': 0.8438453427023884, 'ergodic': 0.8486397166676904, 'divergence': 0.004794373965301995, 'complexity': 0.03703540705484103, 'c2': 0.0013716213757177685, 'alt': 0.04020272079714195, 'alt2': 0.0016162587594929496}


The thing to notice, is here we find that X2 (Chi2) metric is the same for both datasets - which implies we should get the same result. However, because the Chi2 distribution only takes degree's of freedom into account (i.e. the number of states & number of ensembles) and not the number of observations - the resulting p-value is different.

# Making independant results dependant

It's even easier doing this in reverse. You can even use states or ensembles. Here let's use states instead.

Say you start with colors Blue, Green, Salmon Pink, Bright Pink. The results here show that there is no difference between girls and boys (they are independant).

In [9]:
more_states = [
    # Blue, Green, Salmon Pink, Bright Pink
    [100, 20, 75, 75], # boys
    [110, 18, 50, 50]] # girls
chi2(more_states)

Accept H0, they are independant (X2=7.089712585557464,p=0.06909263766557944)

[[100  20  75  75]
 [110  18  50  50]]

0.02515633555997716 ergodic complexity
{'ensemble': 1.2472450176882064, 'ergodic': 1.254376603438616, 'divergence': 0.0071315857504095715, 'complexity': 0.02515633555997716, 'c2': 0.0006328412188061714, 'alt': 0.022461223362639653, 'alt2': 0.0005045065549463893}


And again, let's say you're unethical and this isn't the result you want. You can p-hack the results by grouping the pinks together into a single state. Now changing the result from Independant to Dependant.

In [10]:
fewer_states = [
    # Blue, Green, Pink
    [100, 20, 75*2], # boys
    [110, 18, 50*2]] # girls
chi2(fewer_states)

Reject H0, they are dependant (X2=7.089712585557464,p=0.028872771437316067)

[[100  20 150]
 [110  18 100]]

0.015697144681952067 ergodic complexity
{'ensemble': 0.8992795655998804, 'ergodic': 0.9064111513502902, 'divergence': 0.007131585750409708, 'complexity': 0.015697144681952067, 'c2': 0.00024640035116613607, 'alt': 0.016487622726818627, 'alt2': 0.0002718417031819061}


Again we can see the X2 results are identical (because the same total volumes of observations were used) but the p-values now varied because the degree's of freedom has changed.